In [ ]:
import os
import csv
import sqlite3
import itertools
import time
import math
import statistics
import collections

import igraph
import scipy.special

import matplotlib.pylab as plt
# %matplotlib widget
%matplotlib inline

In [ ]:
g = igraph.Graph.Read_Pickle(input('Input to read igraph pickle: ').strip())
print(f'Nodes: {len(g.vs)}')
print(f'Edges: {len(g.es)}')

In [ ]:
stime = time.time()
partition = g.community_leiden(objective_function='modularity')
etime = time.time()
print(f'{len(partition)} during {etime-stime}')

In [ ]:
l = [len(p) for p in partition if len(p) >= len(g.vs)*0.01]

In [ ]:
fig1 = plt.figure(figsize=(4, 4))
fig1.set_facecolor('white')
ax1 = fig1.add_subplot()
ax1.hist(l)
ax1.set_title('Histogram of cluster size', fontsize='large')
ax1.set_xlabel('Cluster size', fontsize='x-large')
ax1.set_ylabel('Cluster count', fontsize='x-large')
fig1.savefig('histogram.png')

In [ ]:
# Load database
path = 'file:index.db?mode=ro'
conn = sqlite3.connect(path, uri=True)
cur = conn.cursor()


def get_txin_counts(indices):
    counts = list()
    for n in g.vs.select(indices):
        name = n['name']
        cur.execute('''SELECT COUNT(*) FROM TxIn WHERE addr = ?''', (name,))
        tmp_result = cur.fetchone()
        if tmp_result is not None:
            counts.append(tmp_result[0])
        else:
            counts.append(0)
    return counts


def get_txout_counts(indices):
    counts = list()
    for n in g.vs.select(indices):
        name = n['name']
        cur.execute('''SELECT COUNT(*) FROM TxOut WHERE addr = ?''', (name,))
        tmp_result = cur.fetchone()
        if tmp_result is not None:
            counts.append(tmp_result[0])
        else:
            counts.append(0)
    return counts

In [ ]:
with open('meta.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['ClusterSize', 
                                           'TxInMean', 'TxInMedian', 'TxInStdev', 'TxInMin', 'TxInMax', 
                                           'TxOutMean', 'TxOutMedian', 'TxOutStdev', 'TxOutMin', 'TxOutMax'])
    writer.writeheader()
    for i in range(0, len(partition)):
        p = partition[i]
        if len(p) < len(g.vs)*0.01:
            continue
        txin = get_txin_counts(p)
        txout = get_txout_counts(p)
        writer.writerow({'ClusterSize': len(p), 
                         'TxInMean': statistics.mean(txin),
                         'TxInMedian': statistics.median(txin),
                         'TxInStdev': statistics.stdev(txin),
                         'TxInMin': min(txin),
                         'TxInMax': max(txin),
                         'TxOutMean': statistics.mean(txout),
                         'TxOutMedian': statistics.median(txout),
                         'TxOutStdev': statistics.stdev(txout),
                         'TxOutMin': min(txout),
                         'TxOutMax': max(txout)})

In [ ]:
stime = time.time()
rank = g.pagerank()
etime = time.time()
print(f'Max pagerank: {max(rank)} at {g.vs[rank.index(max(rank))]["name"]} during {etime-stime}')

In [ ]:
stime = time.time()
# https://stackoverflow.com/questions/37855553/python-igraph-community-cluster-colors
g2 = partition.cluster_graph(combine_vertices='random', combine_edges='sum')
g2.delete_vertices(g2.vs.select(_degree=0))
layout = g2.layout_drl()
igraph.plot(g2, 'partition.svg', layout=layout, vertex_size=5, edge_width=0.5)
etime = time.time()
print(f'Plotting done during {etime-stime}')

In [ ]:
conn.close()